In [19]:
#packages
import numpy as np
from scipy import constants
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import asyncio
import plotly.io as pio



In [20]:
class KMC_sim:
    def __init__(self, L):
        self.L = L # Lattice size
        self.c_B = 0.5 # Concentration of B atoms
        self.initialize_lattice()

    def initialize_lattice(self):
        num_B_atoms = int(self.L**3 *self.c_B)
        num_A_atoms = int(self.L**3 - num_B_atoms)
        lattice_values = np.array([-1] * num_B_atoms + [1] * num_A_atoms)
        np.random.shuffle(lattice_values)
        self.lattice = lattice_values.reshape((self.L, self.L, self.L))
        
        #Set the center atom to be a vacancy
        self.lattice[self.L//2, self.L//2, self.L//2] = 0
        self.vacancy_position = (self.L//2, self.L//2, self.L//2)

        #create lists and maps to store things
        self.atom_list = []
        self.atom_to_lattice_map = {}
        self.lattice_to_atom_map = {}
        self.atom_to_displacement_map = {}
        for index in range(self.L * self.L):
            i, j, k = index // self.L, index % self.L, (index // self.L) - (index % self.L)
            #still not the right k value - starts off ok, but they get closer and closer together closer to the center
            self.atom_list.append(self.lattice[i, j ,k])
            self.atom_to_lattice_map[index] = (i, j, k)
            self.lattice_to_atom_map[(i, j, k)] = index
            self.atom_to_displacement_map[index] = np.array([0,0,0])

    # Function to calculate Mean Squared Displacement (MSD) with unwrapped periodic boundary conditions
    def calculate_msd(self):
        msd_A = np.mean([np.sum(disp ** 2) for index, disp in self.atom_to_displacement_map.items() if self.atom_list[index] == 1])
        msd_B = np.mean([np.sum(disp ** 2) for index, disp in self.atom_to_displacement_map.items() if self.atom_list[index] == -1])
        msd_vac = np.mean([np.sum(disp ** 2) for index, disp in self.atom_to_displacement_map.items() if self.atom_list[index] == 0])
        return msd_A*9, msd_B*9, msd_vac*9 # Multiply by 9 to get the MSD in terms of lattice spacing (3 Å)    
    
    def initialize_parameters(self, T, E_aa, E_bb, E_ab, e0_a, e0_b, c_B):
        self.T = T
        self.E_aa = E_aa
        self.E_bb = E_bb
        self.E_ab = E_ab
        self.e0_a = e0_a
        self.e0_b = e0_b
        self.c_B = c_B
        self.initialize_lattice()
        #Total energy per atom
        self.total_energy_per_atom = self.get_total_energy()/(self.L**2)
        self.time = 0
    
    #Helper function to calculate energy contribution for an atom and its neighbors
    def calculate_energy_contribution(self, i, j, k):
        atom_type = self.lattice[i, j, k]
        # Neighbors in 6 directions with periodic boundary conditions
        neighbor_position_lists = [((i+1)%self.L, j, k), ((i-1)%self.L, j, k), (i, (j+1)%self.L, k), (i, (j-1)%self.L, k),
                                   (i, j, (k+1%self.L)), (i, j, (k-1%self.L))]
        E = 0
        for ni, nj, nk in neighbor_position_lists:
            neighbor_type = self.lattice[ni, nj, nk]
            if atom_type == 0 or neighbor_type == 0:
                E += 0
            if atom_type == 1 and neighbor_type == 1:
                E += self.E_aa
            elif atom_type == -1 and neighbor_type == -1:
                E += self.E_bb
            else:
                E += self.E_ab
        return E
    
      #bond counting model
    def get_total_energy(self):
        E = 0
        for i in range(self.L):
            for j in range(self.L):
                for k in range(self.L):
                    E += self.calculate_energy_contribution(i,j,k)
        return E/2
    
     #calcuate the energy change if the vacancy and neighbor are swapped
    def get_energy_change(self, vacancy_pos, neighbor_pos):
        i_vac, j_vac, k_vac = vacancy_pos
        i_neigh, j_neigh, k_neigh = neighbor_pos

        dE_before = self.calculate_energy_contribution(i_neigh, j_neigh, k_neigh)
        self.lattice[i_vac, j_vac, k_vac], self.lattice[i_neigh, j_neigh, k_neigh] = self.lattice[i_neigh, j_neigh, k_neigh] , self.lattice[i_vac, j_vac, k_vac]
        dE_after = self.calculate_energy_contribution(i_vac, j_vac)
        self.lattice[i_vac, j_vac, k_vac], self.lattice[i_neigh, j_neigh, k_neigh] = self.lattice[i_neigh, j_neigh, k_neigh] , self.lattice[i_vac, j_vac, k_vac]
        return dE_after - dE_before
    
    def one_simulation_step(self):
        #pick two random atoms: pos_1 = (i1, j1, k1) and pos_2 = (i2, j2, k2)
        kB = constants.value('Boltzmann constant in eV/K')
        i_vac, j_vac, k_vac = self.vacancy_position
        neighbor_position_lists = [((i_vac + 1)% self.L, j_vac, k_vac), ((i_vac - 1)% self.L, j_vac, k_vac),
                                   (i_vac, (j_vac + 1)% self.L, k_vac), (i_vac, (j_vac - 1)%self.L, k_vac),
                                   (i_vac, j_vac, (k_vac + 1)% self.L), (i_vac, j_vac, (k_vac - 1)%self.L)]
        rates = []
        energy_changes = []
        for pos in neighbor_position_lists:
            dE = self.get_energy_change((i_vac, j_vac, k_vac), pos)
            migration_barrier = dE/2 + self.e0_A if self.lattice[pos] == 1 else dE/2+self.e0_b
            rates.append(np.exp(-migration_barrier/(kB*self.T)))
            energy_changes.append(dE)
        rates = np.array(rates)
        probabilities = rates / np.sum(rates)
        chosen_index = np.random.choice(len(neighbor_position_lists), p=probabilities)
        chosen_pos = neighbor_position_lists[chosen_index]
        dE = energy_changes[chosen_index]
        #swap the frequency with the chosen neighbor
        i_neigh, j_neigh, k_neigh = chosen_pos
        self._update_displacement(i_vac, j_vac, k_vac, i_neigh, j_neigh, k_neigh)
        self.lattice[i_vac, j_vac, k_vac], self.lattice[i_neigh, j_neigh, k_neigh] = self.lattice[i_neigh, j_neigh, k_neigh] , self.lattice[i_vac, j_vac, k_vac]
        self.vacancy_position = (i_neigh, j_neigh, k_neigh)
        self.total_energy_per_atom += dE / (self.L **3)
        self.time += np.random.uniform()/(np.sum(rates)*1e13)

    def _unwrap_positions(self, i1, j1, k1, i2, j2, k2):
        #unwrap the periodic boundary conditions
        dx = (i2 - i1 + self.L // 2) % self.L - self.L // 2
        dy = (j2 - j1 + self.L // 2) % self.L - self.L // 2
        dz = (k2 - k1 + self.L // 2) % self.L - self.L // 2
        return dx, dy, dz
    
    def _update_displacement(self, i_old, j_old, k_old, i_new, j_new, k_new):
        old_atom_index = self.lattice_to_atom_map[(i_old, j_old, k_old)]
        new_atom_index = self.lattice_to_atom_map[(i_new, j_new, k_new)]
        dx, dy, dz = self._unwrap_positions(i_old, j_old, k_old, i_new, j_new, k_new)
        self.atom_to_displacement_map[old_atom_index] += np.array([dx, dy, dz])
        self.atom_to_displacement_map[new_atom_index] -= np.array([dx, dy, dz])
        
        #Swap positions in the map
        self.atom_to_lattice_map[old_atom_index], self.atom_to_lattice_map[new_atom_index] = (i_new, j_new, k_new), (i_old, j_old, k_old)
        self.lattice_to_atom_map[(i_old, j_old, k_old)], self.lattice_to_atom_map[(i_new, j_new, k_new)] = new_atom_index, old_atom_index

    def calculate_warren_cowley_sro(self):
        # Calculate the Warren-Cowley short-range order parameter
        # The order parameter is defined as alpha_AB = 1 - (P_AB)/(2*c_A*c_B)
        n_total = self.L*self.L*2 # Total number of atoms
        n_AB = 0
        for i in range(self.L):
            for j in range(self.L):
                for k in range(self.L): # Only count the upper triangle to avoid double counting
                    atom_type = self.lattice[i, j]
                    if atom_type == 0:
                        continue
                # Neighbors in 6 directions with periodic boundary conditions
                neighbor_position_lists = [((i+1)%self.L, j), ((i-1)%self.L, j),
                                           (i, (j+1)%self.L), (i, (j-1)%self.L),
                                           (i, j, (k+1)%self.L)]
                for ni, nj in neighbor_position_lists:
                    neighbor_type = self.lattice[ni, nj]
                    if neighbor_type == 0:
                        continue
                    if atom_type != neighbor_type:
                        n_AB += 1
        p_AB = n_AB/n_total/2
        return 1 - 2*p_AB












In [21]:
#visualization (just for personal practice)
fig = go.Figure()
fig.update_layout(height=750, width=750)



def initialize_plot(lattice):
    type_a_x, type_a_y, type_a_z = np.where(lattice == 1)
    type_b_x, type_b_y, type_b_z = np.where(lattice == -1)
    fig.add_scatter3d(x=type_a_x + 0.5, y=type_a_y + 0.5, z=type_a_z + 0.5, mode='markers', marker_line_width=.1,
                           marker=dict(color='#FFCB05', size=9), name='Atom A')
    fig.add_scatter3d(x=type_b_x + 0.5, y=type_b_y + 0.5, z=type_b_z +0.5, mode='markers', marker_line_width=.1,
                          marker=dict(color='#002743', size=9), name='Atom B')
    
    
    
    
    
    


def updater_plot(lattice, step_list, energy_list, time_list, sro_list,
                 msd_A_list, msd_B_list):
    type_a_x, type_a_y, type_a_z = np.where(lattice == 1)
    type_b_x, type_b_y, type_b_z = np.where(lattice == -1)
    
    with fig.batch_update():
        fig.data[0].x = type_a_x + 0.5
        fig.data[0].y = type_a_y + 0.5
        fig.data[0].z = type_a_z + 0.5
        fig.data[1].x = type_b_x + 0.5
        fig.data[1].y = type_b_y + 0.5
        fig.data[1].z = type_b_z + 0.5
        fig.data[2].x = step_list
        fig.data[2].y = energy_list
        fig.data[3].x = time_list
        fig.data[3].y = energy_list
        fig.data[4].x = time_list
        fig.data[4].y = sro_list
        fig.data[5].x = time_list
        fig.data[5].y = msd_A_list
        fig.data[6].x = time_list
        fig.data[6].y = msd_B_list


# Define the animation function
async def animate_simulation(steps, T, interval, E_aa, E_bb, E_ab, e0_a, e0_b, c_B):
    global stop_animation
    MC.initialize_parameters(T, E_aa, E_bb, E_ab, e0_a, e0_b, c_B)
    step_list = []
    energy_list = []
    time_list = []
    sro_list = []
    msd_a_list = []
    msd_b_list = []
    for step in range(steps+1):
        if stop_animation:
            break
        MC.one_simulation_step()
        if step % interval == 0:
            step_list.append(step)
            energy_list.append(MC.total_energy_per_atom)
            sro_list.append(MC.calculate_warren_cowley_sro())
            time_list.append(MC.time)
            msd_a, msd_b, _ = MC.calculate_msd()
            msd_a_list.append(msd_a)
            msd_b_list.append(msd_b)
            updater_plot(MC.lattice, step_list, energy_list, time_list, sro_list, msd_a_list, msd_b_list)
            step_info.value = f"Current Step: {step}, time: {MC.time:.2e} s, Energy: {MC.total_energy_per_atom:.4f} eV/atom"
        await asyncio.sleep(1/interval/10) #allow interruption

#Callback function for play button
async def on_play_button_clicked(b):
    global stop_animation
    stop_animation = False
    play_button_disabled = True
    await animate_simulation(int(total_steps_input.value), float(temperature_input.value),
                            int(interval_input.value), float(E_aa_input.value),
                            float(E_bb_input.value), float(E_ab_input.value),
                            float(e0_a_input.value), float(e0_b_input.value),
                            float(c_B_slider.value) / 100)

#Callback function for stop button
def on_stop_button_clicked(b):
    global stop_animation
    stop_animation = True

#Callback function for restart button
def on_restart_button_clicked(b):
    on_stop_button_clicked(b)
    MC.c_B = float(c_B_slider.value) / 100
    MC.initialize_lattice()
    updater_plot(MC.lattice, [], [], [], [], [], [])
    step_info.value = "Lattice restarted"
    play_button.disabled = False

# Create interactive widgets with layout adjustments
style = {'description_width': 'initial'}
temperature_input = widgets.Text(value='700', description='Temperature (K):', layout=widgets.Layout(width='300px'), style=style)
total_steps_input = widgets.Text(value=f'{int(1e6)}', description='Total Steps:', layout=widgets.Layout(width='300px'), style=style)
interval_input = widgets.Text(value='100', description='Display Interval:', layout=widgets.Layout(width='300px'), style=style)
E_aa_input = widgets.Text(value='-1.0', description='E_AA (eV):', layout=widgets.Layout(width='300px'), style=style)
E_bb_input = widgets.Text(value='-1.0', description='E_BB (eV):', layout=widgets.Layout(width='300px'), style=style)
E_ab_input = widgets.Text(value='-0.9', description='E_AB (eV):', layout=widgets.Layout(width='300px'), style=style)
e0_a_input = widgets.Text(value='0.8', description='e0_A (eV):', layout=widgets.Layout(width='450px'), style=style)
e0_b_input = widgets.Text(value='0.6', description='e0_B (eV):', layout=widgets.Layout(width='450px'), style=style)
c_B_slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Concentration B (%):', 
                               layout=widgets.Layout(width='900px'),style=style)
play_button = widgets.Button(description="Play", layout=widgets.Layout(width='300px'))
stop_button = widgets.Button(description="Stop", layout=widgets.Layout(width='300px'))
restart_button = widgets.Button(description="Restart", layout=widgets.Layout(width='300px'))
step_info = widgets.Label(value=f"Lattice initialized.", layout=widgets.Layout(width='1000px'))

play_button.on_click(lambda b: asyncio.ensure_future(on_play_button_clicked(b)))
stop_button.on_click(on_stop_button_clicked)
restart_button.on_click(on_restart_button_clicked)

# Arrange widgets in a more organized layout
inputs_box = widgets.VBox([
    widgets.HBox([temperature_input,total_steps_input,interval_input]),
    widgets.HBox([E_aa_input, E_bb_input, E_ab_input]),
    widgets.HBox([e0_a_input, e0_b_input]),
    widgets.HBox([c_B_slider]),
    widgets.HBox([play_button, stop_button,restart_button]),
    step_info
    ])

    



#Finally make the figure (maybe)
MC = KMC_sim(7)
for i in range(1000):
    MC.one_simulation_step()
    if i % 100 == 0:
        print(MC.time, MC.get_total_energy(), MC.calculate_msd())



AttributeError: 'KMC_sim' object has no attribute 'E_bb'